In [1]:
import librosa
import os
import numpy as np

current_dir = os.getcwd()

In [2]:
y, sr = librosa.load(os.path.join(current_dir, "Audio_Files\\UK_OST\\03SandsOfTide.mp3"))

In [3]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

In [4]:
y_harmonic, y_percussive = librosa.effects.hpss(y)

In [5]:
# MFCC is p much the intensity of the different audio bands
# This is huge bc it's a lot of frames
hop_length = 512
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
mfcc_delta = librosa.feature.delta(mfcc)
# Represents the changein MFCC per 
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

In [6]:
# This is cool, it's the like relative strength of pitches
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

In [10]:
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [11]:
# This flattens the matrix into a vector we can pass into our model
# Do it by beats tbh, frames is a LOT. At least for now.
# The first 12 features are the chromagram - the intensity of sound in semitones from C to Bb
beat_features_flattened = beat_features.T.flatten()